In [133]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.tools import job_monitor

In [128]:
def FourierAdder(reg1_qubits, reg2_qubits):
    '''
        Functionality:
            This function creates a QuantumCircuit object that contatins the necessary gates to perform {|reg1> + |reg2> (mod |reg2>)} in Fourier basis.
        Parameters:
            reg1_qubits (int) - Number of qubits in `reg1`.
            reg2_qubits (int) - Number of qubits in `reg2`.
        Returns:
            qc (QuantumCircuit object) - A quantum circuit with the gates needed to write the result into `reg2`.
    '''
    
    # Initializing
    reg1 = QuantumRegister(reg1_qubits, 'reg1')
    reg2 = QuantumRegister(reg2_qubits, 'reg2')
    qc = QuantumCircuit(reg1, reg2)
    qc.name = 'Fourier addition'
    
    # Applying the controlled phase shifts to create addition
    for control_q in range(reg1_qubits):
        for target_q in range(reg2_qubits):
            k = reg2_qubits - target_q
            phase = (2 * np.pi * (2 ** control_q)) / (2 ** k)
            if phase == 2 * np.pi: # Phase shifts of 2pi multiples are indistinguishable = Breaking from the inner loop
                break
            qc.cp(theta = phase, control_qubit = reg1[control_q], target_qubit = reg2[target_q])
            
    return qc

def qft(num_qubits, inverse = False):
    '''
        Functionality:
            This function creates a quantum circuit for QFT of `num_qubits`.
        Parameters:
            num_qubits (int) - The amount of qubits the QFT will be applied to.
            inverse (bool):
                False (default) - A QFT is generated.
                True - A QFT^{dagger} is generated.
        Returns:
            QFT_gate (Gate object) - The QFT as a Gate object to be appended to a quantum circuit.
    '''
    
    # Initalizing circuit
    qc = QuantumCircuit(num_qubits)
    
    # Handling each qubit from the MSB to the LSB (little-endian)
    for i, target_q in reversed(list(enumerate(qc.qubits))):
        qc.h(target_q)
        k = i + 1
        
        for j, control_q in enumerate(qc.qubits[0:i]):    
            phase = ((2 ** j) * (2 * np.pi)) / (2 ** k) 
            qc.cp(theta = phase, control_qubit  = control_q, target_qubit = target_q)

    # Performing final SWAPS
    for i in range(int(num_qubits / 2)):
        qc.swap(i, num_qubits - i - 1)
    
    # Transforming the QuantumCircuit object to a Gate object and returning it
    if inverse:
        qc = qc.inverse()
        QFT_gate = qc.to_gate(label = 'QFT_Dagger')
    else:
        QFT_gate = qc.to_gate(label = 'QFT')
        
    return QFT_gate

def EncodeInteger(i):
    '''
        Functionality:
            This function encodes a positive integer into a quantum state.
        Parameters:
            i (int) - The integer to encode.
        Returns: {'encoded_reg': qc, 'binary': i_bin, 'length': i_len}
            encoded_reg (QuantumCircuit object) - A quantum circuit with the value of `i` encoded within its state.
            binary (str) - The bitstring representation of `i`.
            length (int) - The length of `i_bin`.
    '''
    
    # Translating `i` to binary and measuring its bitstring's length
    i_bin = bin(i)[2:]
    i_len = len(i_bin)
    
    # Initializing circuit
    qc = QuantumCircuit(i_len)
    qc.name = f'Integer encoded: {i}'
    
    # Encoding `i_bin` into the circuit (little-endian)
    for index, d in enumerate(reversed(i_bin)):
        if d == '1':
            qc.x(index)
    
    return {'encoded_reg': qc, 'binary': i_bin, 'length': i_len}

def QuantumMultiply(x, y, backend, shots = 1024):
    '''
        Functionality:
            This function builds a quantum circuit that computes x * y.
        Parameters:
            x (int) - First operand.
            y (int) - Second operand.
            backend (Simulator or IBMQBackend object or None) - The backend to run the circuit upon.
                # If `None` - The function will not run the circuit.
                # NOTE: Current real quantum backends would provide valuable results for circuit depths of not more than few dozens steps.
            shots (int) - Amount of iterations over the experiment, default = 1024.
        Returns: {'qc': qc, 'xy': xy, 'tpqc': tpqc,  'counts': counts}
            qc (QuantumCircuit object) - The quantum circuit that computes x * y (not transpiled).
            xy (int) - The result of x * y (`xy = None` if `backend == None`)
            tpqc (QuantumCircuit object) - The transpiled quantum circuit (optimization_level = 3) with respect to `backend`.
                # `tpqc = None` if `backend = None`.
            counts - (Counts dict object) - The results of running the circuit 1024 times (`None` if `backend = None`).
    '''
    
    # Encoding `x` and `y` into quantum registers and setting registers' lengths
    x_encoded = EncodeInteger(x)
    len_x = x_encoded['length']
    y_encoded = EncodeInteger(y)
    len_y = y_encoded['length']
    len_result = len_x + len_y # That covers the maximum value case where `x` and `y` are full-ones bitstrings
    
    # Initalizing the registers and circuit
    reg_x = QuantumRegister(len_x, 'reg_x')
    reg_y = QuantumRegister(len_y, 'reg_y')
    reg_result = QuantumRegister(len_result, 'reg_result')
    classical_result = ClassicalRegister(len_result, 'classical_result')
    qc = QuantumCircuit(reg_x, reg_y, reg_result, classical_result)
    
    # Setting the `x` and `y` values to their quantum registers
    qc.append(instruction = x_encoded['encoded_reg'], qargs = reg_x)
    qc.append(instruction = y_encoded['encoded_reg'], qargs = reg_y)
    
    # Transforming reg_result to Fourier basis (for the upcoming Fourier addtion)
    qc.h(reg_result)
    qc.barrier()
    
    # x * y = Adding the value of `y` to reg_result `x` times
    c_iteration = FourierAdder(reg1_qubits = len_y, reg2_qubits = len_result).control()
    for i in range(len_x):
        times = 2 ** i
        c_iteration.name = f'If x[{i}] == 1: \nAdding y to result {times} times'
        for iteration in range(times):
            qc.append(instruction = c_iteration, qargs = [reg_x[i]] + reg_y[:] + reg_result[:])
    qc.barrier()

    # Transforming reg_result back to the computational basis
    qc.append(instruction = qft(num_qubits = len_result, inverse = True), qargs = reg_result)
    qc.barrier()
    
    # Measuring
    qc.measure(reg_result, classical_result)

    # If `backend is None` then all the derived variables are `None` also
    if backend is None:
        xy = None
        tpqc = None
        counts = None
    # Running the circuit and processing the results
    else:
        print('Transpiling the circuit..')
        tpqc = transpile(qc, backend, optimization_level = 3)
        print('Running the circuit..')
        job = backend.run(tpqc, shots = shots)
        job_monitor(job) # Monitoring the job's progression and displaying it to the user
        results = job.result()
        counts = results.get_counts()
        common_result = max(counts, key = counts.get) # Taking the most common result (noisy hardware = multiple results)
        xy = int(common_result, 2) # Translating the result from binary to decimal
    
    return {'qc': qc, 'xy': xy, 'tpqc': tpqc, 'counts': counts}

In [ ]:
###### Run this cell ######

# Settings
x = 5
y = 6
shots = 1024

# Backend settings options
backend = Aer.get_backend('aer_simulator')
print(f'The chosen backend is {backend}')

# x * y computation
m = QuantumMultiply(x = x, y = y, backend = backend, shots = shots)

# Output
if backend is not None:
    print(f'\nx * y = {x} * {y} = {m["xy"]}')
    print(f'\nThe transpiled circuit depth is: {m["tpqc"].depth()}')
    print(f'\nThe transpiled circuit\'s gate count is: {m["tpqc"].count_ops()}')
    print(f'\nThe results of running the circuit {shots} times:')
    display(plot_histogram(m['counts']))

print("\nThe high-level circuit:")
display(m['qc'].draw(output = 'mpl'))

###### Run this cell ######
